In [1]:
#IMPORT PACKAGES

#import all the packages are needed for the project.
import pandas as pd
import json
import csv
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce



# Loading in the Json files into 4 seperate dataframes

In [2]:
#OPEN THE FIRST DOCUMENT    /// TIP DATASET
#Read in line by line from the file then load the objects into the 1 dataframe

with open('yelp_academic_dataset_tip.json') as f:
    data2 = pd.DataFrame(json.loads(line) for line in f)

In [95]:
#OPEN THE SECOND DOCUMENT    /// USER DATASET
#Read in line by line from the file then load the objects into the 1 dataframe

with open('yelp_academic_dataset_user.json') as f:
    data3 = pd.DataFrame(json.loads(line) for line in f)

In [ ]:
#OPEN THE THIRD DOCUMENT    /// BUSINESS DATASET
#Read in line by line from the file then load the objects into the 1 dataframe

with open('yelp_academic_dataset_business.json') as f:
    data4 = pd.DataFrame(json.loads(line) for line in f)

In [ ]:
#OPEN THE FOURTH DOCUMENT    /// REVIEW DATASET
#Read in line by line from the file then load the objects into the 1 dataframe

with open('yelp_academic_dataset_review.json') as f:
    data5 = pd.DataFrame(json.loads(line) for line in f)


In [ ]:
#STILL DEADLING WITH THIS 4TH DOCUMENT

#Slice the dataframe to only take 50000/60000 rows out of the whopping 2GB file
data5select = data5.iloc[0:55000,:]

## Function for splitting columns

In [97]:
#CUSTOM FUNCTION 
#TO SPLIT COLUMN'S WHICH HAVE DICTIONARIES
#When loading the json objects the structure is nested so some rows have a dictionary in every cell.
#This function splits these into individual columns

def splitting_row(dataframe, column):  
    """If a column has a dictionary within it this functionn splits it."""
    listuse = []
    for row in dataframe[column]:
        listuse.append(row)
    return listuse

# First Document

In [32]:
data2.to_csv('yelp_tips.csv')

# Second Document

In [98]:
#SECOND DOCUMENT
#Split the compliments row

complimentsdf = pd.DataFrame(splitting_row(data3, 'compliments'))

In [99]:
#SECOND DOCUMENT
#Split the compliments row

votesdf = pd.DataFrame(splitting_row(data3, 'votes'))

In [100]:
#SECOND DOCUMENT
#Concatinate the 2 dataframes above backontu the original dataframe.

result = pd.concat([data3, complimentsdf, votesdf], axis=1, join='inner')

In [101]:
#SECOND DOCUMENT
#Remove the rows that had the dictionaries in the cell as we don't need them anymore

for element in ['friends', 'votes', 'compliments']:
    result = result.drop(element, 1)

In [102]:
result.to_csv('yelp_userdata.csv') 

# Third Document

In [17]:

#THIRD DOCUMENT

attributesdf = pd.DataFrame(splitting_row(data4, 'attributes'))

#THIRD DOCUMENT

for element in ['Parking', 'Good For', 'Ambience']:
    attributesdf = attributesdf.drop(element, 1)

In [5]:
#THIRD DOCUMENT

result4 = pd.concat([data4, attributesdf], axis=1, join='inner')   

In [6]:
#THIRD DOCUMENT

for element in ['attributes', 'hours']:
    result4 = result4.drop(element, 1)

In [10]:
result4.to_csv('yelp_bus.csv')

# Fourth Document

In [18]:
#FOURTH DOCUMENT
#Split the votes row
votes4thdf = pd.DataFrame(splitting_row(data5select, 'votes'))

In [20]:
#FOURTH DOCUMENT
#Concatinate the columns

result5 = pd.concat([data5select, votes4thdf], axis=1, join='inner')

In [21]:
#FOURTH DOCUMENT

#Remove the votes column

result5 = result5.drop('votes', 1)

In [22]:
#FOURTH DOCUMENT

result5.to_csv('yelp_reviews.csv')

# Main Single ABT 

## CSV into DAtaframes

In [2]:
tipsdf = pd.read_csv('yelp_tips.csv')

In [3]:
usersdf = pd.read_csv('yelp_userdata.csv')

In [4]:
bus_df = pd.read_csv('yelp_bus.csv')

/Applications/anaconda/envs/assignmentsecond/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (26,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
reviews_df = pd.read_csv('yelp_reviews.csv')

## FIND OUT THE SHAPE OF THE TABLES TO BE MERGED

### FIRST MERGE; BUSINESS AND REVIEWS TABLE
- The business table has 77445 rows of unique businesses data. Business_id is the identifier
- We will merge on the reviews 'business_ID'
- Reviews has 55,000 rows. Only 2600 businesses. Review_id identifier. Only 2600 businesses mentioned
- We will merge the business and reviews on reviews. 
- The table will be a reviews table with businesses mensioned multiple times. Business info expanded upon

### SECOND MERGE; REVIEWS AND USERS TABLES
- User tables have around 500k rows. 
- Unique values are user_id. This means this table is 500k user information
- We will merge this ontu the first_merge based on user_id

### THIRD MERGE; REVIEWS AND TIPS
- There is 500k tip listed with multiple businesses and multiple users.
- There is only 50k unique businesses mentioned.
- We will come up with an aggregate table of Bus_ID | total tips 
- Then merge that with the above based on business ID
We will be taking an aggregate of the tips 



In [11]:
bus_df.shape

(77445, 47)

In [12]:
print(str(len(bus_df.business_id.unique())))

77445


In [13]:
reviews_df.shape

(55000, 11)

In [14]:
print(str(len(reviews_df.business_id.unique())))

2612


In [15]:
print(str(len(reviews_df.review_id.unique())))

55000


In [16]:
print(str(len(tipsdf.user_id.unique())))

117133


# Merge the Tables

## First Merge

In [6]:
# MERGE
#First try to merge the business table with the reviews table

first_merge = pd.merge(bus_df, reviews_df, how='right', on=['business_id'])

In [7]:
#Check the schema is appropriate and merge was success

usersdf.shape

(552339, 23)

In [21]:
print(str(len(usersdf.user_id.unique())))

552339


In [16]:
first_merge.iloc[:,50:65].head(6)

,stars_y,text,type_y,user_id,cool,funny,useful
0,4,"Mr Hoagie is an institution. Walking in, it do...",review,PUFPaY9KxDAcGqfsorJp3Q,0,0,0
1,5,Excellent food. Superb customer service. I mis...,review,Iu6AxdBYGR4A0wspR9BYHA,0,0,0
2,5,Yes this place is a little out dated and not o...,review,auESFwWvW42h6alXgFxAXQ,0,0,0
3,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,0,0,0
4,3,We checked this place out this past Monday for...,review,I_47G-R2_egp7ME5u_ltew,0,0,0
5,1,Wing sauce is like water. Pretty much a lot of...,review,PP_xoMSYlGr2pb67BbqBdA,0,0,0


## Second merge

In [17]:
second_merge = pd.merge(first_merge, usersdf, how='left', on=['user_id'])

In [26]:
#Check the shape is still appropriote

second_merge.shape

(55000, 79)

In [27]:
print(str(len(second_merge.review_id.unique())))

55000


In [28]:
print(str(len(second_merge.user_id.unique())))

30727


# Manage the duplicate columns

### Duplicate columns
1. Name      
2. Review count       
3. Stars      
4. Unamed      
5. Type   
6. Cool   X Y 1 
7. Funny   X Y 1       
8. Useful    X Y 1


* **Name** = name_x is the name of the bar and name_y is the name of the owner. Rename as such
* **Review count**= review_count_x is the count for the business the y is reviews the user has made. Rename as such
* **Stars** = Stars x is the stars of the business. Stars y is the stars given in the review
* **Unnamed**= This is just indexes we dont need remove this rows
* **Type** Type x and type y both have a cardinality of 1 so remove these rows
* **Cool, Funny, Useful** X- This is related to users rating about the review itself while Y is the compliments by the user while cool1 is votes by th user. 

In [25]:
second_merge.rename(columns={'name_x': 'Business name', 'name_y': 'Reviewer name',
                             'review_count_x': 'Business review count', 'review_count_y': 'Reviewer review count',
                            'stars_x': 'Business stars', 'stars_y': 'Reviewer Rating',
                             'name_x': 'Business name', 'name_y': 'Reviewer name',
                            }, inplace=True)

In [26]:
for element in ['type_x', 'type_y', 'Unnamed: 0', 'Unnamed: 0_x', 'Unnamed: 0_y']:
    second_merge = second_merge.drop(element, 1)

In [27]:
second_merge.rename(columns={'cool_x': 'Review = cool', 'funny_x': 'Review = funny', 'useful_x' : 'Review = useful',
                             'cool_y': 'Compliment cool', 'funny_y': 'Compliment funny', 
                             'cool.1': 'Votes cool', 'funny.1': 'Votes funny', 'useful_y' : 'Votes useful'
                            }, inplace=True)

In [28]:
second_merge.shape

(55000, 74)

# Manage the NAN values

### Problem columns
1. Accepts Insurance  
2. Ages Allowed   
3. BYOB BYOB/Corkage     
4. By Appointment Only   
5. Coat Check   
6. Corkage   
7. Dietary Restrictions   
8. Dogs Alowed    
9. Drive through     
10. Good for dancing    
11. Hair Types Specialised In 
12. Happy Hour    
13. Music    
14. Open 24 Hours    
15. 'Open at counter'  
16. Smoking



In [13]:
#To sho the amount of null values in each column

second_merge.iloc[:,0:40].isnull().sum()

business_id                      0
categories                       0
city                             0
full_address                     0
latitude                         0
longitude                        0
Business name                    0
neighborhoods                    0
open                             0
Business review count            0
Business stars                   0
state                            0
Accepts Credit Cards          7098
Accepts Insurance            54841
Ages Allowed                 54824
Alcohol                      19282
Attire                       20760
BYOB                         47979
BYOB/Corkage                 45303
By Appointment Only          51752
Caters                       25265
Coat Check                   45931
Corkage                      50659
Delivery                     20179
Dietary Restrictions         54347
Dogs Allowed                 48664
Drive-Thru                   52212
Good For Dancing             45906
Good For Groups     

In [14]:
second_merge.iloc[:,39:76].isnull().sum()

Smoking                  45921
Take-out                 20687
Takes Reservations       20669
Waiter Service           21230
Wheelchair Accessible    17108
Wi-Fi                    19546
date                         0
review_id                    0
Reviewer Rating              0
text                         0
user_id                      0
Review = cool                0
Review = funny               0
Review = useful              0
average_stars                0
elite                        0
fans                         0
Reviewer name                0
Reviewer review count        0
type                         0
yelping_since                0
Compliment cool          27687
cute                     45335
Compliment funny         34238
hot                      31757
list                     46700
more                     34952
note                     27640
photos                   40678
plain                    26338
profile                  44431
writer                   30056
Votes co

In [29]:
#Anything that has around 40k+ NAN remove that column

for element in ['Accepts Insurance', 'Ages Allowed', 'BYOB', 'BYOB/Corkage', 'By Appointment Only', 'Coat Check',   
'Corkage', 'Dietary Restrictions', 'Dogs Allowed', 'Drive-Thru', 'Good For Dancing', 'Hair Types Specialized In', 
'Happy Hour', 'Music', 'Open 24 Hours', 'Order at Counter', 'Smoking']:
    second_merge = second_merge.drop(element, 1)



In [16]:
#Now check the shape of the bad boy

second_merge.shape

(55000, 57)

In [18]:
second_merge.isnull().sum()

business_id                  0
categories                   0
city                         0
full_address                 0
latitude                     0
longitude                    0
Business name                0
neighborhoods                0
open                         0
Business review count        0
Business stars               0
state                        0
Accepts Credit Cards      7098
Alcohol                  19282
Attire                   20760
Caters                   25265
Delivery                 20179
Good For Groups          18733
Good for Kids            16312
Has TV                   20017
Noise Level              20125
Outdoor Seating          18022
Price Range               7486
Take-out                 20687
Takes Reservations       20669
Waiter Service           21230
Wheelchair Accessible    17108
Wi-Fi                    19546
date                         0
review_id                    0
Reviewer Rating              0
text                         0
user_id 

In [30]:
play_df = second_merge

In [31]:
#Drop *ROWS* that have Null values in the subset below
#Subset is all the business attributes as they are what where missing the most

df4 = play_df.dropna(subset=['Accepts Credit Cards', 'Alcohol' , 'Attire', 'Caters', 'Delivery', 'Good For Groups',       
'Good for Kids', 'Has TV', 'Noise Level', 'Outdoor Seating', 'Price Range', 'Take-out', 'Takes Reservations',
'Waiter Service', 'Wheelchair Accessible', 'Wi-Fi']) 

          


In [32]:
df4.shape

(25924, 57)

In [33]:
df4.isnull().sum()

business_id                  0
categories                   0
city                         0
full_address                 0
latitude                     0
longitude                    0
Business name                0
neighborhoods                0
open                         0
Business review count        0
Business stars               0
state                        0
Accepts Credit Cards         0
Alcohol                      0
Attire                       0
Caters                       0
Delivery                     0
Good For Groups              0
Good for Kids                0
Has TV                       0
Noise Level                  0
Outdoor Seating              0
Price Range                  0
Take-out                     0
Takes Reservations           0
Waiter Service               0
Wheelchair Accessible        0
Wi-Fi                        0
date                         0
review_id                    0
Reviewer Rating              0
text                         0
user_id 

In [37]:
#Make the columns from the compliments dictionary clear

df4.rename(columns={'cute': 'Compliments cute', 'hot': 'Compliments hot',
                             'list': 'Compliments list', 'more': 'Compliments more',
                            'note': 'Compliments note', 'photos': 'Compliments photos',
                             'plain': 'Complimetns plain', 'profile': 'Compliments profile',
                             'writer' : 'Compliments writer'
                            }, inplace=True)

/Applications/anaconda/envs/assignmentsecond/lib/python3.5/site-packages/pandas/core/frame.py:2697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [38]:
#WRITE IT TO THE .CSV

df4.to_csv('ABT.csv')

In [39]:
df4.shape

(25924, 57)

In [41]:
df4.isnull().sum()

business_id                  0
categories                   0
city                         0
full_address                 0
latitude                     0
longitude                    0
Business name                0
neighborhoods                0
open                         0
Business review count        0
Business stars               0
state                        0
Accepts Credit Cards         0
Alcohol                      0
Attire                       0
Caters                       0
Delivery                     0
Good For Groups              0
Good for Kids                0
Has TV                       0
Noise Level                  0
Outdoor Seating              0
Price Range                  0
Take-out                     0
Takes Reservations           0
Waiter Service               0
Wheelchair Accessible        0
Wi-Fi                        0
date                         0
review_id                    0
Reviewer Rating              0
text                         0
user_id 

In [ ]:
#for element in ['Accepts Insurance', 'Ages Allowed', 'BYOB', 'BYOB/Corkage', 'By Appointment Only', 'Coat Check',   
#'Corkage', 'Dietary Restrictions', 'Dogs Allowed', 'Drive-Thru', 'Good For Dancing', 'Hair Types Specialized In', 
#'Happy Hour', 'Music', 'Open 24 Hours', 'Order at Counter', 'Smoking']:
#    second_merge = second_merge.drop(element, 1)
    

# Data Quality Report

## 